In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HD").getOrCreate()

In [2]:
data=spark.read.csv("../data/hack_data.csv",inferSchema=True,header=True)

In [4]:
data.describe().toPandas

,summary,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
0,count,334,334,334,334,334,334,334
1,mean,30.008982035928145,607.2452694610777,0.5119760479041916,5.258502994011977,10.838323353293413,None,57.342395209580864
2,stddev,14.088200614636158,286.33593163576757,0.5006065264451406,2.30190693339697,3.06352633036022,None,13.41106336843464
3,min,1.0,10.0,0,1.0,6.0,Afghanistan,40.0
4,max,60.0,1330.5,1,10.0,15.0,Zimbabwe,75.0


In [11]:
data.select("Location").show() #looks like this variable won't be useful since it has mostly distinct values

+--------------------+
|            Location|
+--------------------+
|            Slovenia|
|British Virgin Is...|
|             Tokelau|
|             Bolivia|
|                Iraq|
|    Marshall Islands|
|             Georgia|
|         Timor-Leste|
|Palestinian Terri...|
|          Bangladesh|
|Northern Mariana ...|
|            Zimbabwe|
|         Isle of Man|
|Sao Tome and Prin...|
|              Greece|
|     Solomon Islands|
|       Guinea-Bissau|
|        Burkina Faso|
|            Mongolia|
|             Nigeria|
+--------------------+
only showing top 20 rows



In [19]:
from pyspark.ml.feature import VectorAssembler
sindex=StringIndexer(inputCol="Location")
assembler=VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted','WPM_Typing_Speed'],outputCol='features')

In [14]:
final_data=assembler.transform(data)

In [16]:
from pyspark.ml.clustering import KMeans
kmeans2 = KMeans(k=2, seed=1)
kmeans3=KMeans(k=3, seed=1)
model2 = kmeans2.fit(final_data)
model3=kmeans3.fit(final_data)

In [17]:
results2=model2.transform(final_data)
results3=model3.transform(final_data)

In [18]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator(metricName='silhouette', distanceMeasure='squaredEuclidean')

silhouette2 = evaluator.evaluate(results2)
silhouette3 = evaluator.evaluate(results3)
print("Silhouette with 2 clusters = " + str(silhouette2))
print("Silhouette with 3 clusters = " + str(silhouette3))

Silhouette with 2 clusters = 0.8048521975748283
Silhouette with 3 clusters = 0.6958546319645178


It seems it is more likely that the _attacks_ have been perpetrated by only **two hackers**. The **third suspect** is probably _**innocent**_.

### With Location feature 

In [ ]:
from pyspark.ml.feature import VectorAssembler,StringIndexer
sindex=StringIndexer(inputCol="Location",outputCol="Location_ind")
assembler=VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted','Location_ind','WPM_Typing_Speed'],outputCol='features')

In [20]:
data_ind=sindex.fit(data).transform(data)
final_data=assembler.transform(data_ind)

In [21]:
kmeans2 = KMeans(k=2, seed=1)
kmeans3=KMeans(k=3, seed=1)
model2 = kmeans2.fit(final_data)
model3=kmeans3.fit(final_data)

In [22]:
results2=model2.transform(final_data)
results3=model3.transform(final_data)

In [23]:
silhouette2 = evaluator.evaluate(results2)
silhouette3 = evaluator.evaluate(results3)
print("Silhouette with 2 clusters = " + str(silhouette2))
print("Silhouette with 3 clusters = " + str(silhouette3))

Silhouette with 2 clusters = 0.7880858006977343
Silhouette with 3 clusters = 0.6562678799064662


Same outcome with worse evaluation results. Location was indeed an useless feature.